### Linear and Multilinear Functions

In [2]:
from kingdon import Algebra
from gc_utils import *
import numpy as np
# use vanilla algebra, so we're not bother by the sign of I*I.reverse()
alg = Algebra(4) 
locals().update(alg.blades)

In [3]:
#1.1 linear transformation
A = create_random_multivector(alg)
B = create_random_multivector(alg)
C = create_random_multivector(alg)
D = create_random_multivector(alg)
x = create_random_multivector(alg).grade(1)
a = create_random_multivector(alg).grade(1)
f = lambda x: (A*x*B+C*x*D).grade(1)
differential(f, x, a), f(a)

(0.484 𝐞₁ + 0.408 𝐞₂ + 0.618 𝐞₃ + 0.618 𝐞₄,
 0.484 𝐞₁ + 0.408 𝐞₂ + 0.618 𝐞₃ + 0.618 𝐞₄)

In [4]:
#1.2
g = lambda x: (C*x*D).grade(1)
g(f(a)), differential(g, f(a), f(a))

(-0.771 𝐞₁ + -0.171 𝐞₂ + 1.11 𝐞₃ + -0.5 𝐞₄,
 -0.771 𝐞₁ + -0.171 𝐞₂ + 1.11 𝐞₃ + -0.5 𝐞₄)

In [5]:
# modify the input of the original skew_symmetrizer: a multivector Ar instead of vectors
def skew_symmetrizer(F, Ar, alg, h=1e-6, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(alg.frame)
    drF = 0
    r = Ar.grades[0]
    for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
        # if F is linear, vectors cause no difference in vectors_partial. The input is just Ar.
        # So actually we allow Ar being any multivector
        drF += (Ar.sp(wedge(base_vectors[::-1]))) * vectors_partial(F, np.zeros(r), reci_vectors, h=h)
    return (1/factorial(r)) * drF

In [6]:
#1.4 Extend a vector function to a multivector function.
# For linear f we ignore the position where f is measured, 
# just the differential at each direction.
def outermorphism(f, A: MultiVector, alg, h=1e-6, frame=None, r_frame=None):
    wf = lambda vectors: wedge([f(v) for v in vectors])
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        outer += skew_symmetrizer(wf, A.grade(r), alg, h, frame, r_frame)
    return outer

outermorphism(f, x, alg), f(x)

(-0.35 𝐞₁ + 0.419 𝐞₂ + 4.12 𝐞₃ + -1.38 𝐞₄,
 -0.35 𝐞₁ + 0.419 𝐞₂ + 4.12 𝐞₃ + -1.38 𝐞₄)

In [7]:
#1.6 A frame can be arbitrary as long as it represent the same space. 
# It's pretty handy to specify a domain.
h = 1e-2
A2 = e12
A2frame = [e1, e2]
A2r_frame = reciprocal(A2frame)

outermorphism(f, P(A, A2), alg, h, frame=A2frame, r_frame=A2r_frame), outermorphism(f, A, alg, h, frame=A2frame, r_frame=A2r_frame)

(0.408 + 0.0355 𝐞₁ + 0.371 𝐞₂ + 2.07 𝐞₃ + 0.0683 𝐞₄ + -0.12 𝐞₁₂ + -0.567 𝐞₁₃ + -0.0167 𝐞₁₄ + 1.1 𝐞₂₃ + 0.0567 𝐞₂₄ + 0.115 𝐞₃₄,
 0.408 + 0.0355 𝐞₁ + 0.371 𝐞₂ + 2.07 𝐞₃ + 0.0683 𝐞₄ + -0.12 𝐞₁₂ + -0.567 𝐞₁₃ + -0.0167 𝐞₁₄ + 1.1 𝐞₂₃ + 0.0567 𝐞₂₄ + 0.115 𝐞₃₄)

In [8]:
#1.7 An outermorphism preserves outer products
vectors = create_r_vectors(3, alg)
outermorphism(f, wedge(vectors), alg), wedge([f(v) for v in vectors])

(-1.18 𝐞₁₂₃ + 0.243 𝐞₁₂₄ + -0.494 𝐞₁₃₄ + -0.993 𝐞₂₃₄,
 -1.18 𝐞₁₂₃ + 0.243 𝐞₁₂₄ + -0.494 𝐞₁₃₄ + -0.993 𝐞₂₃₄)

In [9]:
# By 1.7, splitting A into simple blades, we get an easier implement of 1.4
# Here, the value at v replaces the slope at direction v by linearity
def outermorphism_(f, A: MultiVector, alg):
    outer = A.grade(0)
    for vectors in multi_frame_vectors(alg.frame):
        Vr = wedge(reciprocal(vectors)[::-1])
        outer += alg.sp(Vr,A) * wedge([f(v) for v in vectors]) # frame split of A
    return outer

outermorphism_(f, D, alg), outermorphism(f, D, alg)

(0.133 + -0.758 𝐞₁ + 0.632 𝐞₂ + 2.46 𝐞₃ + -1.75 𝐞₄ + 2.99 𝐞₁₂ + -3.55 𝐞₁₃ + 2.07 𝐞₁₄ + 10.3 𝐞₂₃ + -7.8 𝐞₂₄ + -1.99 𝐞₃₄ + 1.84 𝐞₁₂₃ + -0.0322 𝐞₁₂₄ + 2.91 𝐞₁₃₄ + 1.28 𝐞₂₃₄ + -0.595 𝐞₁₂₃₄,
 0.133 + -0.758 𝐞₁ + 0.632 𝐞₂ + 2.46 𝐞₃ + -1.75 𝐞₄ + 2.99 𝐞₁₂ + -3.55 𝐞₁₃ + 2.07 𝐞₁₄ + 10.3 𝐞₂₃ + -7.8 𝐞₂₄ + -1.99 𝐞₃₄ + 1.84 𝐞₁₂₃ + -0.0322 𝐞₁₂₄ + 2.91 𝐞₁₃₄ + 1.28 𝐞₂₃₄ + -0.595 𝐞₁₂₃₄)

In [10]:
#1.9 anoter outer product property
outermorphism(f, A.reverse(), alg), outermorphism(f, A, alg).reverse()

(0.408 + -0.253 𝐞₁ + 0.52 𝐞₂ + 3.0 𝐞₃ + -0.737 𝐞₄ + -0.19 𝐞₁₂ + -0.225 𝐞₁₃ + 0.0719 𝐞₁₄ + -1.79 𝐞₂₃ + 2.18 𝐞₂₄ + 3.36 𝐞₃₄ + 0.439 𝐞₁₂₃ + -0.645 𝐞₁₂₄ + -2.62 𝐞₁₃₄ + 2.99 𝐞₂₃₄ + -1.02 𝐞₁₂₃₄,
 0.408 + -0.253 𝐞₁ + 0.52 𝐞₂ + 3.0 𝐞₃ + -0.737 𝐞₄ + -0.19 𝐞₁₂ + -0.225 𝐞₁₃ + 0.0719 𝐞₁₄ + -1.79 𝐞₂₃ + 2.18 𝐞₂₄ + 3.36 𝐞₃₄ + 0.439 𝐞₁₂₃ + -0.645 𝐞₁₂₄ + -2.62 𝐞₁₃₄ + 2.99 𝐞₂₃₄ + -1.02 𝐞₁₂₃₄)

In [11]:
#1.10
fA2 = lambda x: P(f(x), A2)
outermorphism(fA2, A, alg), P(outermorphism(fA2, A, alg), A2)

(0.408 + -0.253 𝐞₁ + 0.52 𝐞₂ + 0.19 𝐞₁₂,
 0.408 + -0.253 𝐞₁ + 0.52 𝐞₂ + 0.19 𝐞₁₂)

In [12]:
#1.11
def adjoint_outermorphism(f, A, alg, h=1e-6, frame=None, r_frame=None):
    F = lambda vectors: wedge([f(v) for v in vectors]).sp(A)
    outer = 0
    for r in A.grades:
        if r == 0:
           outer += A.grade(0)
           continue
        # Why np.zeros(r)? The derivative of a linear function is constant
        outer += simplicial_derivative(F, np.zeros(r), alg, h, frame, r_frame)
    return outer

In [13]:
#1.12
adjoint_outermorphism(fA2, P(A, A2), alg), adjoint_outermorphism(fA2, A, alg)

(0.408 + 1.27 𝐞₁ + -0.463 𝐞₂ + 0.794 𝐞₃ + -0.203 𝐞₄ + -0.12 𝐞₁₂ + -0.409 𝐞₁₃ + 0.28 𝐞₁₄ + 0.225 𝐞₂₃ + -0.122 𝐞₂₄ + 0.11 𝐞₃₄,
 0.408 + 1.27 𝐞₁ + -0.463 𝐞₂ + 0.794 𝐞₃ + -0.203 𝐞₄ + -0.12 𝐞₁₂ + -0.409 𝐞₁₃ + 0.28 𝐞₁₄ + 0.225 𝐞₂₃ + -0.122 𝐞₂₄ + 0.11 𝐞₃₄)

In [14]:
#1.13
# Exercise: implement the adjoint outermorphism by spliting A into vectors.
adjoint_outermorphism(f, A^B, alg), adjoint_outermorphism(f, A, alg) ^ adjoint_outermorphism(f, B, alg)

(0.105 + 1.13 𝐞₁ + 0.181 𝐞₂ + 0.992 𝐞₃ + -0.759 𝐞₄ + 1.87 𝐞₁₂ + -0.35 𝐞₁₃ + -0.832 𝐞₁₄ + -1.5 𝐞₂₃ + 0.411 𝐞₂₄ + -0.00677 𝐞₃₄ + 6.74 𝐞₁₂₃ + -12.0 𝐞₁₂₄ + 5.66 𝐞₁₃₄ + 5.78 𝐞₂₃₄ + -4.72 𝐞₁₂₃₄,
 0.105 + 1.13 𝐞₁ + 0.181 𝐞₂ + 0.992 𝐞₃ + -0.759 𝐞₄ + 1.87 𝐞₁₂ + -0.35 𝐞₁₃ + -0.832 𝐞₁₄ + -1.5 𝐞₂₃ + 0.411 𝐞₂₄ + -0.00677 𝐞₃₄ + 6.74 𝐞₁₂₃ + -12.0 𝐞₁₂₄ + 5.66 𝐞₁₃₄ + 5.78 𝐞₂₃₄ + -4.72 𝐞₁₂₃₄)

In [15]:
#1.14a r <= s
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
    adjoint_outermorphism(f, outermorphism(f, Ar, alg) | Bs, alg),
)

(2.66 𝐞₁ + -6.47 𝐞₂ + 3.73 𝐞₃ + -2.08 𝐞₄,
 2.16 𝐞₁ + 3.02 𝐞₂ + -2.02 𝐞₃ + -1.8 𝐞₄,
 2.66 𝐞₁ + -6.47 𝐞₂ + 3.73 𝐞₃ + -2.08 𝐞₄)

In [16]:
#1.14b r >= s
r, s = 3, 2
Ar = A.grade(r)
Bs = B.grade(s)
(
    outermorphism(f, Ar | adjoint_outermorphism(f, Bs, alg), alg),
    Ar | adjoint_outermorphism(f, Bs, alg),
    outermorphism(f, Ar, alg) | Bs,
)

(-1.02 𝐞₁ + 0.869 𝐞₂ + -0.634 𝐞₃ + -0.806 𝐞₄,
 0.0726 𝐞₁ + -0.683 𝐞₂ + -0.341 𝐞₃ + 0.0881 𝐞₄,
 -1.02 𝐞₁ + 0.869 𝐞₂ + -0.634 𝐞₃ + -0.806 𝐞₄)

In [17]:
#1.15 r = s
for r,s in [(1,1), (2,2), (3,3)]:
    Ar = A.grade(r)
    Bs = B.grade(s)
    print(
        Ar | adjoint_outermorphism(f, Bs, alg),
        outermorphism(f, Ar, alg) | Bs,
    )

0.871 0.871
1.28 1.28
1.46 1.46


In [18]:
# proof of 1.14a
r, s = 2, 3
Ar = A.grade(r)
Bs = B.grade(s)
frame = alg.frame
r_frame = reciprocal(alg.frame)
rhs = 0
r = Ar.grades[0]
F = lambda vectors: wedge([f(v) for v in vectors]) 
for base_vectors, reci_vectors in zip(permutations(frame, s), permutations(r_frame, s)):
    rhs += (Ar | (wedge(base_vectors[::-1]))) * (vectors_partial(F, np.zeros(s), reci_vectors, h=h) | Bs)
(
    Ar | adjoint_outermorphism(f, Bs, alg),
    rhs / factorial(s)
)

(2.66 𝐞₁ + -6.47 𝐞₂ + 3.73 𝐞₃ + -2.08 𝐞₄,
 2.66 𝐞₁ + -6.47 𝐞₂ + 3.73 𝐞₃ + -2.08 𝐞₄)

In [19]:
# singular, nonsingular functions
I = alg.pseudoscalar((1,))
# fA2 is singular
outermorphism(fA2, I, alg), outermorphism(f, I, alg), adjoint_outermorphism(f, I, alg)

(, -10.8 𝐞₁₂₃₄, -10.8 𝐞₁₂₃₄)

In [20]:
#1.16 Use inverse instead of reverse for negative signatures. case I = Ip
n = 4
Ir = I.inv()
F = lambda vectors: wedge([f(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame, alg, h=1e-2), 
    Ir * outermorphism(f, I, alg), 
    adjoint_outermorphism(f, I, alg) * Ir
)

(-10.8, -10.8, -10.8)

In [21]:
#1.16 case I != Ip
n = 2
I = e12
Ip = e23
Ir = I.inv()
Ipr = Ip.inv()
h = 1e-2
# a function from I to Ip
g = lambda x: P((A*P(x, I)*B).grade(1), Ip)
F = lambda vectors: wedge([g(v) for v in vectors])
(
    simplicial_derivative(F, alg.frame[:2], alg, h),
    Ir * outermorphism(g, I, alg, h),
    adjoint_outermorphism(g, Ip, alg, h) * Ipr
)

(-1.4 𝐞₁₃, -1.4 𝐞₁₃, -1.4 𝐞₁₃)

In [22]:
#1.17a
outermorphism(g, I, alg, h), Ip

(1.4 𝐞₂₃, 1 𝐞₂₃)

In [23]:
#1.17b Need extra minus sign for negative signature. You can try Algebra(3,1).
adjoint_outermorphism(g, Ip, alg, h), I

(1.4 𝐞₁₂, 1 𝐞₁₂)

In [24]:
# The relative orientation between I and I' can be arbitrary.
# Here we let the parent algebra determine the orientation.
def det(f, I, Ip):
    return (outermorphism(f, I, alg)/Ip)[0]

det(g, I, Ip)

1.4040278358296427

In [25]:
#1.21a null part goes null
C = create_random_multivector(alg)
C_ = outermorphism(g, C, alg)
(1/det(g, I, Ip)) * adjoint_outermorphism(g, C_*Ip, alg) * Ir, C

(0.178 + 0.00219 𝐞₁ + 0.951 𝐞₂ + 0.463 𝐞₁₂,
 0.178 + 0.00219 𝐞₁ + 0.951 𝐞₂ + 0.248 𝐞₃ + 0.614 𝐞₄ + 0.463 𝐞₁₂ + 0.588 𝐞₁₃ + 0.0665 𝐞₁₄ + 0.746 𝐞₂₃ + 0.769 𝐞₂₄ + 0.55 𝐞₃₄ + 0.0634 𝐞₁₂₃ + 0.808 𝐞₁₂₄ + 0.103 𝐞₁₃₄ + 0.877 𝐞₂₃₄ + 0.91 𝐞₁₂₃₄)

In [26]:
#1.21b null part goes null
C_ = create_random_multivector(alg)
C = adjoint_outermorphism(g, C_, alg)
(1/det(g, I, Ip)) * outermorphism(g, C*I, alg) * Ipr, C_

(0.858 + 0.54 𝐞₂ + 0.644 𝐞₃ + 0.507 𝐞₂₃,
 0.858 + 0.288 𝐞₁ + 0.54 𝐞₂ + 0.644 𝐞₃ + 0.64 𝐞₄ + 0.0954 𝐞₁₂ + 0.587 𝐞₁₃ + 0.399 𝐞₁₄ + 0.507 𝐞₂₃ + 0.869 𝐞₂₄ + 0.85 𝐞₃₄ + 0.593 𝐞₁₂₃ + 0.104 𝐞₁₂₄ + 0.808 𝐞₁₃₄ + 0.392 𝐞₂₃₄ + 0.858 𝐞₁₂₃₄)

In [27]:
#2.1 Note that characteristic multivectors are of even grades
r = 3  # try several r
h = 1e-2
vectors = create_r_vectors(r, alg)
F = lambda vectors: wedge([f(v) for v in vectors])
(
    F(vectors),
    outermorphism(f, wedge(vectors), alg, h),
    simplicial_derivative(F, vectors, alg, h), # characteristic multivectors
    derivative(lambda A: outermorphism(f, A, alg, h), wedge(vectors), alg, h, r),
)


(0.972 𝐞₁₂₃ + 0.763 𝐞₁₂₄ + -1.07 𝐞₁₃₄ + -0.978 𝐞₂₃₄,
 0.972 𝐞₁₂₃ + 0.763 𝐞₁₂₄ + -1.07 𝐞₁₃₄ + -0.978 𝐞₂₃₄,
 19.2 + 5.8 𝐞₁₂ + -0.885 𝐞₁₃ + 1.85 𝐞₁₄ + 5.06 𝐞₂₃ + 1.5 𝐞₂₄ + -0.442 𝐞₃₄,
 19.2 + 5.8 𝐞₁₂ + -0.885 𝐞₁₃ + 1.85 𝐞₁₄ + 5.06 𝐞₂₃ + 1.5 𝐞₂₄ + -0.442 𝐞₃₄)

In [28]:
#2.2
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
images = [f(v) for v in frame]
(
    sum(vr * b for b, vr in zip(images, r_frame)),
    derivative(f, x, alg)
)

(-3.37 + 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
 -3.37 + -0.925 𝐞₁ + 3.4 𝐞₂ + 0.953 𝐞₃ + 8.01 𝐞₄ + 3.1 𝐞₁₂ + 1.29 𝐞₁₃ + 7.23 𝐞₁₄ + 2.87 𝐞₂₃ + -0.0279 𝐞₂₄ + 2.18 𝐞₃₄ + -1.17 𝐞₁₂₃ + 0.203 𝐞₁₂₄ + -2.84 𝐞₁₃₄ + 5.53 𝐞₂₃₄ + 4.76 𝐞₁₂₃₄)

In [29]:
#2.2 comes from linearity
images, [differential(f, a, v) for v in frame]

([0.457 𝐞₁ + 3.09 𝐞₂ + 1.52 𝐞₃ + 0.789 𝐞₄,
  1.45 𝐞₁ + 1.25 𝐞₂ + 0.501 𝐞₃ + 1.65 𝐞₄,
  0.542 𝐞₁ + 0.915 𝐞₂ + 0.925 𝐞₃ + -0.718 𝐞₄,
  0.801 𝐞₁ + 1.86 𝐞₂ + -0.0648 𝐞₃ + 0.334 𝐞₄],
 [0.457 𝐞₁ + 3.09 𝐞₂ + 1.52 𝐞₃ + 0.789 𝐞₄,
  1.45 𝐞₁ + 1.25 𝐞₂ + 0.501 𝐞₃ + 1.65 𝐞₄,
  0.542 𝐞₁ + 0.915 𝐞₂ + 0.925 𝐞₃ + -0.718 𝐞₄,
  0.801 𝐞₁ + 1.86 𝐞₂ + -0.0648 𝐞₃ + 0.334 𝐞₄])

In [30]:
#2.3 trace
def trace(f, alg):
    return sum(vr.sp(f(v)) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
def divergence(f, alg):
    return sum(vr | f(v) for v, vr in zip(alg.frame, reciprocal(alg.frame)))
(
    derivative(f, e1, alg).grade(0),
    trace(f, alg),
    divergence(f, alg)
)

(-3.37, -3.37, -3.37)

In [31]:
#2.4
r = 2
h = 1e-2
F = lambda vectors: wedge([f(v) for v in vectors])
def char_multi_linear(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += wedge(r_vectors[::-1]) * wedge([f(v) for v in vectors])
    return sdf

(
    char_multi_linear(f, r, alg),
    simplicial_derivative(F, np.zeros(r), alg, h)
)

(-2.84 + -2.61 𝐞₁₂ + -2.3 𝐞₁₃ + -2.55 𝐞₁₄ + -1.46 𝐞₂₃ + 0.513 𝐞₂₄ + 0.104 𝐞₃₄ + -0.517 𝐞₁₂₃₄,
 -2.84 + -2.61 𝐞₁₂ + -2.3 𝐞₁₃ + -2.55 𝐞₁₄ + -1.46 𝐞₂₃ + 0.513 𝐞₂₄ + 0.104 𝐞₃₄ + -0.517 𝐞₁₂₃₄)

In [32]:
#2.6 when full rank, only scalar remains
def trans2matrix(f, alg):
    values = [f(a) for a in alg.frame]
    return np.array([[ar.sp(v)[0] for v in values] for ar in reciprocal(alg.frame)])
matrix = trans2matrix(f, alg)
I = alg.pseudoscalar((1,))
char_multi_linear(f, 4, alg)[0], np.linalg.det(matrix), det(f, I, I)

(-15.713906881867167, -15.713906881867173, -15.713906881867162)

In [33]:
#2.7 sum
r = 3
h = lambda x: f(x)+g(x)

# Sometimes I use a random frame to check frame independence
frame = create_r_vectors(len(alg.signature), alg)
r_frame = reciprocal(frame)
drF = 0
coef = [1/(factorial(r-s)*factorial(s)) for s in range(r+1)]
for base_vectors, reci_vectors in zip(permutations(frame, r), permutations(r_frame, r)):
    gh = 0
    for s in range(r+1):
        gh += coef[s]*wedge([f(v) for v in reci_vectors[:s]]) ^ wedge([g(v) for v in reci_vectors[s:]])
    drF += wedge(base_vectors[::-1]) * gh
char_multi_linear(h, r, alg), drF

(54.6 + 25.6 𝐞₁₂ + 8.29 𝐞₁₃ + 7.39 𝐞₁₄ + 12.3 𝐞₂₃ + -1.9 𝐞₂₄ + 2.3 𝐞₃₄,
 54.6 + 25.6 𝐞₁₂ + 8.29 𝐞₁₃ + 7.39 𝐞₁₄ + 12.3 𝐞₂₃ + -1.9 𝐞₂₄ + 2.3 𝐞₃₄)

In [34]:
#2.8 characteristic polynomial
def char_scalar(f, r, alg: Algebra, frame=None, r_frame=None):
    if not frame:
        frame = alg.frame
        r_frame = reciprocal(frame)
    r_vec_frame = r_vector_frame_vectors(frame, r)
    r_reci_frame = r_vector_frame_vectors(r_frame, r, reverse=True)
    sdf = 0
    for vectors, r_vectors in zip(r_vec_frame, r_reci_frame):
        sdf += alg.sp(wedge(r_vectors[::-1]), wedge([f(v) for v in vectors]))[0]
    return sdf

# This calculates n characteristic multivectors. 
# But drops the nonscalar information
def char_poly_coefs(f, alg, frame=None, r_frame=None):
    n = alg.d
    if frame:
        n = len(frame)
    return [(-1)**(n-s) * char_scalar(f, s, alg, frame, r_frame) for s in range(n+1)]

def polynomial(l, coefs):
    return sum(l ** i * coef for i, coef in enumerate(coefs[::-1]))

l = 3.3
n = 4
(
    char_multi_linear(lambda x: f(x) - l*x, n, alg), 
    polynomial(l, char_poly_coefs(f, alg))
)

(1.29e+02, 129.38591582787262)

In [35]:
def r_fold(f,r,value):
    return reduce(lambda v, _: f(v), range(r), value)

r_fold(f, 3, x), f(f(f(x))), r_fold(f, 0, x), x

(9.75 𝐞₁ + 6.83 𝐞₂ + 8.89 𝐞₃ + 4.95 𝐞₄,
 9.75 𝐞₁ + 6.83 𝐞₂ + 8.89 𝐞₃ + 4.95 𝐞₄,
 0.45 𝐞₁ + 0.832 𝐞₂ + 0.108 𝐞₃ + 0.741 𝐞₄,
 0.45 𝐞₁ + 0.832 𝐞₂ + 0.108 𝐞₃ + 0.741 𝐞₄)

In [36]:
#2.9 Cayley Hamilton
sum(r_fold(f, i, x) * coef for i, coef in enumerate(char_poly_coefs(f, alg)[::-1]))

-7.11e-15 𝐞₁ + 3.55e-15 𝐞₂ + 3.55e-15 𝐞₃ + 1.07e-14 𝐞₄

In [37]:
#2.10 We can find eigenvalues like the usual many-fold method.
# How about eigenvectors? 
# What else can we get from characteristic multivector? Some papers I should go through...
matrix = trans2matrix(f, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
coefs = char_poly_coefs(f, alg)
eigvalues, np.roots(coefs)

(array([ 2.38183674+0.j       , -1.31289598+0.j       ,
        -2.21722457+0.3301247j, -2.21722457-0.3301247j]),
 array([ 2.38183674+0.j       , -2.21722457+0.3301247j,
        -2.21722457-0.3301247j, -1.31289598+0.j       ]))

In [38]:
# From Lasenby's paper, we can reconstruct a rotor from characteristic multivector:
# Note that the rotor must have non-zero scalar part
# the paper suggest such method is pretty robust for noisy data
R = blade_exp(create_r_blade(2, alg)) * blade_exp(create_r_blade(2, alg))
rotor = lambda x: R.sw(x) # a screw motion

terms_ratio(R.reverse(), sum(char_multi_linear(rotor, s, alg) for s in range(n+1)))

array([0.14026346, 0.14026346, 0.14026346, 0.14026346, 0.14026346,
       0.14026346, 0.14026346, 0.14026346])

In [39]:
# recovering matrix to transfromation
def matrix2trans(M, alg:Algebra ):
    d = alg.d
    assert M.shape[0] == d, 'dimension not fit'
    return lambda x: sum(c*e for c,e in zip(np.dot(M, x.asfullmv()[1:d+1]), alg.frame))
    
mf = matrix2trans(matrix, alg)
mf(x), f(x)

(1.62 𝐞₁ + 1.28 𝐞₂ + 1.79 𝐞₃ + 0.786 𝐞₄,
 1.62 𝐞₁ + 1.28 𝐞₂ + 1.79 𝐞₃ + 0.786 𝐞₄)

In [40]:
#2.11
s = 2
rhs = (1/s) * sum((-1)**(r+1) * char_scalar(f, s-r, alg) * trace(lambda x: r_fold(f, r, x), alg)[0] for r in range(1,s+1))
char_scalar(f, s, alg), rhs

(-2.8422002264115793, -2.842200226411581)

In [41]:
#2.13
k = 3
char_scalar(f, k, alg), sum(np.prod(comb) for comb in combinations(eigvalues, k))

(19.238482212880626, (19.238482212880612+0j))

In [42]:
#2.14 It's easier to show the eigenvalues of f^k are {lambda^k_i} by linearity
trace(lambda x: r_fold(f, k, x), alg)[0], sum(l**k for l in eigvalues)

(-9.100823431763676, (-9.100823431763738+0j))

In [43]:
# 3.4 A space must bound f to own an invariant subspace.
I = alg.pseudoscalar((1,))
outermorphism(f, I, alg), det(f, I, I)*I, adjoint_outermorphism(f, I, alg)

(-15.7 𝐞₁₂₃₄, -15.7 𝐞₁₂₃₄, -15.7 𝐞₁₂₃₄)

In [44]:
#3.6
l = 2.2
n = 4
F = lambda x: f(x) - l*x
coefs = char_poly_coefs(f, alg)
outermorphism(F, I, alg), polynomial(l, coefs) * I

(-12.5 𝐞₁₂₃₄, -12.5 𝐞₁₂₃₄)

In [45]:
#3.7
I = e12
frame = [e1,e2]
g = lambda x: P((A*x.grade(1)*B).grade(1), I)
coefs = char_poly_coefs(g, alg, frame, frame)
F = lambda x: g(x) - l*x
outermorphism(F, I, alg), polynomial(l, coefs) * I

(6.97 𝐞₁₂, 6.97 𝐞₁₂)

In [46]:
#3.8
I = e123
frame2 = [e3]
frame3 = [e1,e2,e3]
h = lambda x: P((A*P(x, e12).grade(1)*B).grade(1), e12) + P((B*P(x, e3).grade(1)*C).grade(1), e3)
F = lambda x: h(x) - l*x
(
    char_scalar(F, 2, alg, frame, frame) * char_scalar(F, 1, alg, frame2, frame2),
    char_scalar(F, 3, alg, frame3, frame3)
)

(-19.683951866566243, -19.683951866566243)

In [47]:
outermorphism(h, e1, alg), adjoint_outermorphism(h, e1, alg)

(-0.854 𝐞₁ + 1.29 𝐞₂, -0.854 𝐞₁ + 1.65 𝐞₂)

In [48]:
#3.9a
adjoint_outermorphism(h, e123, alg) | e12, outermorphism(h, e12, alg).e12 * outermorphism(h, -e3, alg)

(-0.912 𝐞₃, -0.912 𝐞₃)

In [49]:
#3.9b
outermorphism(h, e123, alg) | e3, adjoint_outermorphism(h, e3, alg).e3 * adjoint_outermorphism(h, e12, alg)

(0.912 𝐞₁₂, 0.912 𝐞₁₂)

In [50]:
def separate_indices(numbers):
    real_indices = []
    conjugate_pairs_indices = []
    visited = set()  # To keep track of indices already processed

    for i, num in enumerate(numbers):
        if np.isreal(num):  # Check if the number is real
            real_indices.append(i)
        elif i not in visited:  # Check if the index is not already processed
            conjugate = np.conj(num)
            for j in range(i + 1, len(numbers)):
                if numbers[j] == conjugate:
                    conjugate_pairs_indices.append((i, j))
                    visited.add(i)
                    visited.add(j)
                    break

    return real_indices, conjugate_pairs_indices

separate_indices(eigvalues), eigvalues

(([0, 1], [(2, 3)]),
 array([ 2.38183674+0.j       , -1.31289598+0.j       ,
        -2.21722457+0.3301247j, -2.21722457-0.3301247j]))

In [51]:
# An example of non proper blade:
def linalg_eigblades(f, alg: Algebra):
    matrix = trans2matrix(f, alg)
    eigvalues, eigvecs = np.linalg.eig(matrix)
    real_indices, conjugate_pairs_indices = separate_indices(eigvalues)
    eigblades = [alg.vector(np.real(eigvecs[:, i])) for i in real_indices]
    for k, l in conjugate_pairs_indices:
        v1 = np.real(eigvecs[:,k] + eigvecs[:,l])
        v2 = np.imag(eigvecs[:,k] - eigvecs[:,l])
        eigblades.append(alg.vector(v1) ^ alg.vector(v2))
    return eigblades 

def terms_ratio(A, B: MultiVector):
    valid_keys = [k for k in B.keys() if not np.isclose(B[k], 0)]
    return np.divide([A[k] for k in valid_keys], [B[k] for k in valid_keys])

eigblades = linalg_eigblades(f, alg)

# right eigenblades but not left!
for blade in eigblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )

[2.38183674 2.38183674 2.38183674 2.38183674] [4.22046201 1.79664144 1.58471985 2.15136733]
[-1.31289598 -1.31289598 -1.31289598 -1.31289598] [-2.19925402 -1.31864721  2.71001585 -0.50701392]
[5.02506712 5.02506712 5.02506712 5.02506712 5.02506712 5.02506712] [   4.01361982    5.11801164    5.83626287    4.90743006    4.63526294
 -515.29360583]


In [52]:
#3.11 reciprocal for blades
def reciprocal(blades):
    I = wedge(blades)
    dualblades = []
    for k in range(len(blades)):
        sign = (-1) ** (blades[k].grades[0] * sum(blade.grades[0] for blade in blades[:k]))
        dualblades.append(sign * wedge(blades[:k] + blades[k+1:]) * I.inv())
    return dualblades

dualblades = reciprocal(eigblades)
# The reciprocal of a right eigenblade is a left eigenblade! Note that they correspond to the same eigenvalues
for blade in dualblades:
    print(
        terms_ratio(outermorphism(f, blade, alg), blade), 
        terms_ratio(adjoint_outermorphism(f, blade, alg), blade)
        )


[1.42000709 2.95135454 4.63326947 3.00114156] [2.38183674 2.38183674 2.38183674 2.38183674]
[ 0.54773198 -2.05728781 -1.52309997 -2.25835544] [-1.31289598 -1.31289598 -1.31289598 -1.31289598]
[-7.67248032e+00  1.15517564e+01  2.30323348e+04  6.04233585e+00
  6.05296557e+00  3.67231321e+00] [5.02506712 5.02506712 5.02506712 5.02506712 5.02506712 5.02506712]


In [53]:
#3.12 
[eigblades[i] | dualblades[j] for i,j in [(0,0), (0,1), (1,1)]]

[1.0, 5.55e-17, 1.0]

In [54]:
# What transformation have a non-trivial proper blade?
# A invariant subspace, say e123 of e1234
# from 3.12 we know we're looking for blades equal to thier own reciprocal
# that means orthogonal to other eigenspaces

pf = lambda x: P(f(P(x, e123)), e123) + P(f(P(x, e4)), e4)
[(outermorphism(pf, blade, alg), adjoint_outermorphism(pf, blade, alg)) for blade in [e123, e4]]

[(5.32 𝐞₁₂₃, 5.32 𝐞₁₂₃), (-1.24 𝐞₄, -1.24 𝐞₄)]

In [55]:
eigblades = linalg_eigblades(pf, alg)
for E in [e123, e4]:
    for blade in eigblades: # not just vectors
        blade = (blade | E)*E
        om = outermorphism(pf, blade, alg)
        am = adjoint_outermorphism(pf, blade, alg)
        print(
            terms_ratio(om, blade), 
            terms_ratio(am, blade)
            )

[1.49606267 1.49606267 1.49606267] [2.5175652  0.90774184 0.79584074]
[] []
[3.5533932 3.5533932 3.5533932] [7.5938814  5.23692092 2.24765873]
[] []
[-1.24275945] [-1.24275945]
[] []


In [56]:
#4.1b A matrix from a symmetric transformation is symmetric
fp = lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2
trans2matrix(fp, alg), fp(x), derivative(lambda x: x | f(x)/2, x, alg, grade=1)

(array([[-0.11462069,  1.57141154,  0.55433424,  0.85834624],
        [ 1.57141154, -0.66261643,  0.87152067,  1.30157094],
        [ 0.55433424,  0.87152067, -1.3455118 ,  1.13249189],
        [ 0.85834624,  1.30157094,  1.13249189, -1.24275945]]),
 1.95 𝐞₁ + 1.21 𝐞₂ + 1.67 𝐞₃ + 0.672 𝐞₄,
 1.95 𝐞₁ + 1.21 𝐞₂ + 1.67 𝐞₃ + 0.672 𝐞₄)

In [57]:
# a curl is a part of a derivative
derivative(f, x, alg, grade=1).grade(2), curl(f, x, alg, grade=1)

(0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄)

In [58]:
#4.1c a skew transfromation leads to a skew matrix
fm = lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2
trans2matrix(fm, alg), fm(x), x | derivative(f, x, alg, grade=1).grade(2)/2

(array([[-6.93889390e-18, -1.03378284e-01, -6.36686931e-01,
         -2.35540093e-01],
        [ 1.03378284e-01, -1.11022302e-16,  3.71220457e-01,
         -3.45114373e-02],
        [ 6.36686931e-01, -3.71220457e-01, -1.11022302e-16,
          1.92411351e-01],
        [ 2.35540093e-01,  3.45114373e-02, -1.92411351e-01,
         -2.22044605e-16]]),
 -0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄,
 -0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄)

In [59]:
#4.2 fp curl free
a | fp(x), x | fp(a), curl(fp, x, alg, grade=1)

(1.86,
 1.86,
 -2.33e-10 𝐞₁₂ + -2.33e-10 𝐞₁₃ + -1.16e-10 𝐞₁₄ + -1.16e-10 𝐞₂₃ + -1.75e-10 𝐞₂₄)

In [60]:
#4.3 spectral decomposition of the symmetric part
matrix = trans2matrix(fp, alg)
eigvalues, eigvecs = np.linalg.eigh(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
mf(x), fp(x)

(1.95 𝐞₁ + 1.21 𝐞₂ + 1.67 𝐞₃ + 0.672 𝐞₄,
 1.95 𝐞₁ + 1.21 𝐞₂ + 1.67 𝐞₃ + 0.672 𝐞₄)

In [61]:
# QR factorization
transframe = [fp(e) for e in alg.frame] #another basis
gs_trans = gram_schmidt(transframe)
gs_trans2 = gram_schmidt([fp(v / norm(v)) for v in gs_trans[::-1]])[::-1]
gs_trans2


[-1.21e+02 𝐞₁ + 4.14e-15 𝐞₂ + -1.14e-14 𝐞₃ + 5.38e-15 𝐞₄,
 -9.55e-16 𝐞₁ + -27.9 𝐞₂ + -1.57e-15 𝐞₃ + -3.52e-15 𝐞₄,
 6.13e-16 𝐞₁ + 3.66e-16 𝐞₂ + -6.5 𝐞₃ + -1.25e-15 𝐞₄,
 -8.33e-17 𝐞₁ + 2.36e-16 𝐞₂ + 3.61e-16 𝐞₃ + -1.87 𝐞₄]

In [62]:
# A GA version of eigenblade-solver given the eigenvalues
def null_space(f, alg, tol=1e-5, frame=None):
    if not frame:
        frame = alg.frame
    null_sp = wedge(frame)
    images = [f(a) for a in frame]
    for v in images:
        contracted = v | null_sp
        if normsq(contracted) >= tol:
            null_sp = contracted
    return null_sp

def eigenblades_from_values(f, eigvalues, alg, frame=None):
    eigblades = []
    for e in eigvalues:
        null_f = lambda x: f(x) - e*x
        eigblades.append(null_space(null_f, alg, frame=frame))
    return eigblades

In [63]:
# One thing to note for a symmetric transformation:
# the characteristic multivectors are of scalar values
# So all subspaces are invariant under the transfromation.
[char_multi_linear(fp, r, alg) for r in range(5)]

[1,
 -3.37 + -2.22e-16 𝐞₁₂ + -2.22e-16 𝐞₁₃ + -2.22e-16 𝐞₂₄ + -4.44e-16 𝐞₃₄,
 -3.49 + 1.78e-15 𝐞₁₂ + 1.78e-15 𝐞₁₄ + 8.88e-16 𝐞₂₄ + 4.44e-16 𝐞₃₄ + 1.11e-16 𝐞₁₂₃₄,
 20.1 + -2.66e-15 𝐞₁₄ + 1.78e-15 𝐞₃₄,
 -15.1]

In [64]:
np.roots(char_poly_coefs(fp, alg)), eigvalues

(array([ 2.4478458 , -2.52236322, -2.16346244, -1.12752851]),
 array([-2.52236322, -2.16346244, -1.12752851,  2.4478458 ]))

In [65]:
eigblades = eigenblades_from_values(fp, eigvalues, alg)
[e/norm(e) for e in eigblades], eigvecs

([0.0875 𝐞₁ + -0.378 𝐞₂ + -0.504 𝐞₃ + 0.772 𝐞₄,
  0.425 𝐞₁ + -0.722 𝐞₂ + 0.544 𝐞₃ + -0.0461 𝐞₄,
  -0.687 𝐞₁ + -0.0035 𝐞₂ + 0.571 𝐞₃ + 0.449 𝐞₄,
  -0.582 𝐞₁ + -0.58 𝐞₂ + -0.352 𝐞₃ + -0.448 𝐞₄],
 [0.0875 𝐞₁ + -0.378 𝐞₂ + -0.504 𝐞₃ + 0.772 𝐞₄,
  -0.425 𝐞₁ + 0.722 𝐞₂ + -0.544 𝐞₃ + 0.0461 𝐞₄,
  0.687 𝐞₁ + 0.0035 𝐞₂ + -0.571 𝐞₃ + -0.449 𝐞₄,
  -0.582 𝐞₁ + -0.58 𝐞₂ + -0.352 𝐞₃ + -0.448 𝐞₄])

In [66]:
#4.4 modify mf so to have a repeat eigenvalue (degeneracy)
eigvalues[1] = eigvalues[0]
mf = lambda x: sum(e * P(x, v) for e, v in zip(eigvalues, eigvecs))
for blade in [eigvecs[0], wedge(eigvecs[:2])]:
    of = outermorphism(mf, blade, alg)
    print(terms_ratio(of, blade))

[-2.52236322 -2.52236322 -2.52236322 -2.52236322]
[6.36231621 6.36231621 6.36231621 6.36231621 6.36231621 6.36231621]


In [67]:
eigblades = eigenblades_from_values(mf, set(eigvalues), alg)
[e/norm(e) for e in eigblades], [wedge(eigvecs[:2])] + eigvecs[2:]

([-0.582 𝐞₁ + -0.58 𝐞₂ + -0.352 𝐞₃ + -0.448 𝐞₄,
  -0.0976 𝐞₁₂ + -0.262 𝐞₁₃ + 0.332 𝐞₁₄ + 0.569 𝐞₂₃ + -0.574 𝐞₂₄ + 0.397 𝐞₃₄,
  -0.687 𝐞₁ + -0.0035 𝐞₂ + 0.571 𝐞₃ + 0.449 𝐞₄],
 [-0.0976 𝐞₁₂ + -0.262 𝐞₁₃ + 0.332 𝐞₁₄ + 0.569 𝐞₂₃ + -0.574 𝐞₂₄ + 0.397 𝐞₃₄,
  0.687 𝐞₁ + 0.0035 𝐞₂ + -0.571 𝐞₃ + -0.449 𝐞₄,
  -0.582 𝐞₁ + -0.58 𝐞₂ + -0.352 𝐞₃ + -0.448 𝐞₄])

In [68]:
#4.6 a reflection is an orthogonal symmetric transformation 
A3 = create_r_blade(3, alg).normalized()
matrix = trans2matrix(lambda x: (-1)**3*(A3.sw(x)), alg)
# a symmetric matrix, orthogonal, unit eigvals
matrix, [np.dot(matrix[i], matrix[j]) for i,j in [(1,1), (1,2), (2,3), (3,3)]], np.linalg.eigvals(matrix)

(array([[-0.95288128,  0.18808455, -0.23791979,  0.00596964],
        [ 0.18808455, -0.24921986, -0.94970829,  0.0238291 ],
        [-0.23791979, -0.94970829,  0.20134483, -0.0301429 ],
        [ 0.00596964,  0.0238291 , -0.0301429 , -0.99924369]]),
 [1.0, 9.031404096804252e-17, 3.469446951953614e-18, 1.0],
 array([-1.+0.00000000e+00j,  1.+0.00000000e+00j, -1.+3.46944695e-18j,
        -1.-3.46944695e-18j]))

In [69]:
#4.6 Find blade A from an orthogonal-symmetric-transfromation?
ot = matrix2trans(matrix, alg)
eigvalues, eigvecs = np.linalg.eig(matrix)
eigvecs = [alg.vector(v) for v in eigvecs.transpose()]
A3_ = wedge([v for v, e in zip(eigvecs, eigvalues) if np.isclose(e, -1)])
(-1)**3*(A3.sw(x)), ot(x)

(-0.294 𝐞₁ + -0.208 𝐞₂ + -0.898 𝐞₃ + -0.721 𝐞₄,
 -0.294 𝐞₁ + -0.208 𝐞₂ + -0.898 𝐞₃ + -0.721 𝐞₄)

In [70]:
#4.7 Not true for whole space, but the proper ones
# In matrix terms: diagonal parts commute
eigblades = linalg_eigblades(pf, alg)
eigblades.append(e123)
eigblades, [max_diff(pf(blade.sw(x)), blade.sw(pf(x))) for blade in eigblades]

([0.613 𝐞₁ + 0.694 𝐞₂ + 0.379 𝐞₃,
  1.0 𝐞₄,
  0.386 𝐞₁₂ + -0.969 𝐞₁₃ + 1.29 𝐞₂₃,
  1 𝐞₁₂₃],
 [0.12025538803755675, 0.0, 2.6542098073788383, 0.0])

In [71]:
#4.8 skew transformation -> skew bilinear form -> bivector
fm_biform = lambda vectors: vectors[0] | fm(vectors[1])
F = simplicial_derivative(fm_biform, np.zeros(2), alg)
a | fm(x), a| (x | F)

(0.0459, 0.0459)

In [72]:
#4.9 so fm(x) is always orthogonal to x
fm(x), x|F, fm(x) | x

(-0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄,
 -0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄,
 -1.25e-16)

In [73]:
#4.10 F is a bivector
derivative(fm, x, alg, grade=1), curl(fm, x, alg, grade=1), curl(f, x, alg, grade=1), 2*F

(-5.82e-11 + 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄)

In [74]:
#4.11 bivector split commutes, you can think about a screw motion splits into translation and rotation
e13*e24, e24*e13

(-1 𝐞₁₂₃₄, -1 𝐞₁₂₃₄)

In [75]:
#4.13 How to factor F?
def wedge_power(A, r):
    if r == 0:
        return 1
    result = A
    for _ in range(r-1):
        result = A ^ result
    return result

Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(3)]
Ck, [char_multi_linear(fm, r, alg) for r in range(1, 5)] # these are full story, but we just take a fraction

([1.0,
  0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
  -0.129 𝐞₁₂₃₄],
 [-4.51e-16 + 0.207 𝐞₁₂ + 1.27 𝐞₁₃ + 0.471 𝐞₁₄ + -0.742 𝐞₂₃ + 0.069 𝐞₂₄ + -0.385 𝐞₃₄,
  0.648 + -1.73e-18 𝐞₁₂ + -3.75e-16 𝐞₁₃ + -1.67e-16 𝐞₁₄ + 8.67e-17 𝐞₂₃ + 7.98e-17 𝐞₃₄ + -0.517 𝐞₁₂₃₄,
  -1.7e-16 + 0.0498 𝐞₁₂ + 0.00892 𝐞₁₃ + 0.096 𝐞₁₄ + -0.0609 𝐞₂₃ + 0.165 𝐞₂₄ + -0.0267 𝐞₃₄,
  0.0167])

In [76]:
[alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]

[0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 -0.0249 𝐞₁₂ + -0.00446 𝐞₁₃ + -0.048 𝐞₁₄ + 0.0305 𝐞₂₃ + -0.0823 𝐞₂₄ + 0.0134 𝐞₃₄]

In [77]:
set.union(*(set(ck.keys()) for ck in Ck if isinstance(ck, MultiVector)))

{0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15}

In [78]:
x|F, fm(x), x|F|F, fm(fm(x))

(-0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄,
 -0.33 𝐞₁ + 0.0612 𝐞₂ + 0.12 𝐞₃ + 0.114 𝐞₄,
 -0.11 𝐞₁ + 0.00659 𝐞₂ + -0.211 𝐞₃ + -0.0987 𝐞₄,
 -0.11 𝐞₁ + 0.00659 𝐞₂ + -0.211 𝐞₃ + -0.0987 𝐞₄)

In [79]:
#4.14 4.18 Why square of eigen values? 
# The odd grade characteristic multivectors have zero scalar
# A correction on Lasenby's paper, not square of f, but the even outermorphism f_(x, y)
Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(3)]
char_poly = char_poly_coefs(fm, alg)
Lk = np.roots(Ck2)

Ck2, char_poly, Lk, np.sqrt(-np.array(Lk)), np.roots(char_poly), char_poly_coefs(lambda x: fm(fm(x)), alg)

([1.0, 0.6475542487470467, 0.01671886172307832],
 [1,
  4.510281037539698e-16,
  0.6475542487470467,
  1.6967763999398144e-16,
  0.016718861723078324],
 array([-0.62061507, -0.02693918]),
 array([0.78779126, 0.1641316 ]),
 array([-9.28429128e-17+0.78779126j, -9.28429128e-17-0.78779126j,
        -1.32671139e-16+0.1641316j , -1.32671139e-16-0.1641316j ]),
 [1,
  1.2951084974940938,
  0.4527642285165089,
  0.021652739885987496,
  0.00027952033731541367])

In [80]:
Ck[1], Ck[1] | Ck[2]

(0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 -0.0249 𝐞₁₂ + -0.00446 𝐞₁₃ + -0.048 𝐞₁₄ + 0.0305 𝐞₂₃ + -0.0823 𝐞₂₄ + 0.0134 𝐞₃₄)

In [81]:
#4.15
mv_map1 = {}
mv_map2 = {}
ck1, ck2 = Ck[1], Ck[1] | Ck[2]
for key in ck2.keys():
    if np.isclose(ck1[key],0) and np.isclose(ck2[key],0):
        continue
    ans = np.linalg.solve(np.array([[1, 1], Lk]), [ck1[key], ck2[key]])
    mv_map1[key], mv_map2[key] = ans
F1 = alg.multivector(mv_map1)
F2 = alg.multivector(mv_map2)
F1 + F2, F, F1*F2, F2*F1, F**2, F1**2, F2**2


(0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 1.73e-18 + 1.04e-17 𝐞₁₂ + 4.34e-18 𝐞₁₃ + -1.56e-17 𝐞₁₄ + 1.19e-17 𝐞₂₃ + 6.07e-18 𝐞₂₄ + -2.08e-17 𝐞₃₄ + -0.129 𝐞₁₂₃₄,
 1.73e-18 + -1.13e-17 𝐞₁₂ + -3.47e-18 𝐞₁₃ + 1.56e-17 𝐞₁₄ + -1.04e-17 𝐞₂₃ + -6.07e-18 𝐞₂₄ + 2.17e-17 𝐞₃₄ + -0.129 𝐞₁₂₃₄,
 -0.648 + -5.2e-18 𝐞₁₂ + -0.259 𝐞₁₂₃₄,
 -0.0269 + 1.63e-19 𝐞₁₄ + -3.47e-18 𝐞₁₂₃₄,
 -0.621 + 9.37e-17 𝐞₁₂₃₄)

In [82]:
#4.16
def normsq(A):
    return A.reverse().sp(A)[0]
(
    [(-normsq(F) - np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2,
    (-normsq(F) + np.sqrt((normsq(F)**2 - normsq(F ^ F))))/2],
    Lk
)


([-0.620615067609407, -0.02693918113763971], array([-0.62061507, -0.02693918]))

In [83]:
#4.17
F2_ = F / (1 + (1/2/Lk[0])*(F ^ F))
F1_ = F / (1 + (1/2/Lk[1])*(F ^ F))
F1_, F1, F2_, F2

(0.0372 𝐞₁₂ + -0.0214 𝐞₁₃ + 0.0702 𝐞₁₄ + -0.0345 𝐞₂₃ + 0.137 𝐞₂₄ + -0.0138 𝐞₃₄,
 0.0372 𝐞₁₂ + -0.0214 𝐞₁₃ + 0.0702 𝐞₁₄ + -0.0345 𝐞₂₃ + 0.137 𝐞₂₄ + -0.0138 𝐞₃₄,
 0.0662 𝐞₁₂ + 0.658 𝐞₁₃ + 0.165 𝐞₁₄ + -0.337 𝐞₂₃ + -0.103 𝐞₂₄ + -0.179 𝐞₃₄,
 0.0662 𝐞₁₂ + 0.658 𝐞₁₃ + 0.165 𝐞₁₄ + -0.337 𝐞₂₃ + -0.103 𝐞₂₄ + -0.179 𝐞₃₄)

In [84]:
def bivector_split(F, alg: Algebra):
    m = alg.d // 2
    if m <= 1:
        return F
    Ck = [(1/factorial(r))*wedge_power(F, r) for r in range(m+1)]
    Ck2 = [(-1)**(2 - i) * alg.sp(Ck[i], Ck[i])[0] for i in range(m+1)]
    Lk = np.roots(Ck2)
    mv_map_list = [{} for _ in range(m)]
    ck_inner_list = [alg.ip(Ck[i-1], Ck[i]) for i in range(1, len(Ck))]
    mv_keys = set.union(*(set(ck.keys()) for ck in ck_inner_list if isinstance(ck, MultiVector)))
    for key in mv_keys:
        if all([np.isclose(ck[key],0) for ck in ck_inner_list]):
            continue
        ans = np.linalg.solve(np.array([np.ones(m), Lk]), [ck[key] for ck in ck_inner_list])
        for i, v in enumerate(ans):
            mv_map_list[i][key] = v
    return [alg.multivector(mv_map) for mv_map in mv_map_list]

In [85]:
# What if we use the above function to split a bivector that's almost a blade?
# The result blades are R itself and its dual with a small magnitude
# Should assert not simple before process?

R = create_r_blade(2, alg) + 1e-12 * A # a blade with a small perturbation
blades = bivector_split(R, alg)
R**2, [terms_ratio(b, r) for r in [R, R.dual()] for b in blades]

(-0.0954 + 5.79e-14 𝐞₁ + 4.28e-14 𝐞₂ + -5.45e-14 𝐞₃ + -7.11e-14 𝐞₄ + -1.28e-14 𝐞₁₂ + 8.37e-14 𝐞₁₃ + -1.31e-13 𝐞₁₄ + 1.47e-13 𝐞₂₃ + -3.24e-14 𝐞₂₄ + -1.41e-13 𝐞₃₄ + -2.92e-14 𝐞₁₂₃ + 5.93e-14 𝐞₁₂₄ + -5.66e-14 𝐞₁₃₄ + -1.18e-13 𝐞₂₃₄ + -1.06e-13 𝐞₁₂₃₄,
 [array([-1.75660800e-12, -3.15256193e-13,  6.62719172e-13,  4.64475974e-13,
         -9.76629689e-13, -1.75352338e-13]),
  array([1., 1., 1., 1., 1., 1.]),
  array([5.54811826e-13, 5.54856267e-13, 5.54862483e-13, 5.54762924e-13,
         5.54897864e-13, 5.55188814e-13]),
  array([-0.31584271, -1.76001703,  0.83725129,  1.19438454, -0.56817632,
         -3.16613294])])

In [86]:
# double skew is symmetric
fm2 = lambda x: fm(fm(x))
curl(fm2, x, alg, h=1e-2, grade=1)

5.22e-15 𝐞₁₂ + 1.42e-14 𝐞₁₃ + 6.22e-15 𝐞₁₄ + -6.66e-15 𝐞₂₃ + 7.77e-15 𝐞₂₄ + -1.78e-15 𝐞₃₄

In [87]:
#4.19
v = P(e1, F1)
fm2(v), (F1 ** 2) * v

(-0.00676 𝐞₁ + -0.0104 𝐞₂ + -0.000315 𝐞₃ + 0.0054 𝐞₄,
 -0.00676 𝐞₁ + -0.0104 𝐞₂ + -0.000315 𝐞₃ + 0.0054 𝐞₄ + 6.95e-19 𝐞₁₂₃ + -2.19e-20 𝐞₁₂₄ + -1.34e-18 𝐞₁₃₄ + 8.71e-19 𝐞₂₃₄)

In [88]:
#4.20 
F1_eigvecs= [v.normalized() for v in gram_schmidt(blade_split(F1, alg))]
F1, np.sqrt(normsq(F1)) * wedge(F1_eigvecs)


(0.0372 𝐞₁₂ + -0.0214 𝐞₁₃ + 0.0702 𝐞₁₄ + -0.0345 𝐞₂₃ + 0.137 𝐞₂₄ + -0.0138 𝐞₃₄,
 0.0372 𝐞₁₂ + -0.0214 𝐞₁₃ + 0.0702 𝐞₁₄ + -0.0345 𝐞₂₃ + 0.137 𝐞₂₄ + -0.0138 𝐞₃₄)

In [89]:
# If Lk[0] == Lk[1], the linear equation has no unique solution in 4.15
# nonunique bivector split? We should find another split of e12 + e34. 
t = 1.1
R = blade_exp(t*e13) * blade_exp(t*e24)
a,b,c,d = [R.sw(v) for v in alg.frame]
a,b,c,d, a*b+c*d, a*b, c*d

(-0.589 𝐞₁ + -0.808 𝐞₃,
 -0.589 𝐞₂ + -0.808 𝐞₄,
 0.808 𝐞₁ + -0.589 𝐞₃,
 0.808 𝐞₂ + -0.589 𝐞₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 𝐞₁₂ + 0.476 𝐞₁₄ + -0.476 𝐞₂₃ + 0.654 𝐞₃₄,
 0.654 𝐞₁₂ + -0.476 𝐞₁₄ + 0.476 𝐞₂₃ + 0.346 𝐞₃₄)

In [90]:
R.sw(e12 + e34), (e12+e34) * (R**2), R**2

(1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 1.0 𝐞₁₂ + 1.0 𝐞₃₄,
 0.346 + -0.476 𝐞₁₃ + -0.476 𝐞₂₄ + -0.654 𝐞₁₂₃₄)

In [91]:
#4.21
B2 = B.grade(2)
# fm = lambda x: f(x) - adjoint_outermorphism(f, x, alg)
# F = curl(fm, x, alg, grade=1) / 2
outermorphism(fm, B2, alg), (0.5 * (B2 | (F^F))) - ((B2|F) * F)

(0.0759 𝐞₁₂ + 0.14 𝐞₁₃ + 0.119 𝐞₁₄ + -0.0868 𝐞₂₃ + -0.0735 𝐞₂₄ + -0.0266 𝐞₃₄,
 0.0759 𝐞₁₂ + 0.14 𝐞₁₃ + 0.119 𝐞₁₄ + -0.0868 𝐞₂₃ + -0.0735 𝐞₂₄ + -0.0266 𝐞₃₄)

In [92]:
# What is a symmetric bivector function?
# I guess it means:
outermorphism(fm, B2, alg), adjoint_outermorphism(fm, B2, alg)

(0.0759 𝐞₁₂ + 0.14 𝐞₁₃ + 0.119 𝐞₁₄ + -0.0868 𝐞₂₃ + -0.0735 𝐞₂₄ + -0.0266 𝐞₃₄,
 0.0759 𝐞₁₂ + 0.14 𝐞₁₃ + 0.119 𝐞₁₄ + -0.0868 𝐞₂₃ + -0.0735 𝐞₂₄ + -0.0266 𝐞₃₄)

In [93]:
#4.23 The bivector split solves the eigenbivector problem of a skew transformation
outermorphism(fm, F1, alg), normsq(F1)*F1

(0.001 𝐞₁₂ + -0.000576 𝐞₁₃ + 0.00189 𝐞₁₄ + -0.000928 𝐞₂₃ + 0.00369 𝐞₂₄ + -0.000371 𝐞₃₄,
 0.001 𝐞₁₂ + -0.000576 𝐞₁₃ + 0.00189 𝐞₁₄ + -0.000928 𝐞₂₃ + 0.00369 𝐞₂₄ + -0.000371 𝐞₃₄)

In [94]:
#5.1 normal transformation
af = lambda x: adjoint_outermorphism(f, x, alg)

fp(fm(x)), fm(fp(x)), af(f(x)), f(af(x)) # not normal

(0.298 𝐞₁ + -0.306 𝐞₂ + -0.162 𝐞₃ + -0.209 𝐞₄,
 -1.35 𝐞₁ + 0.798 𝐞₂ + 0.922 𝐞₃ + 0.181 𝐞₄,
 4.94 𝐞₁ + 3.48 𝐞₂ + -0.216 𝐞₃ + 4.02 𝐞₄,
 1.65 𝐞₁ + 5.69 𝐞₂ + 1.95 𝐞₃ + 4.8 𝐞₄)

In [95]:
M = blade_exp(create_r_blade(2, alg))
Mdual = blade_split(M.grade(2).dual(), alg)
rigid = lambda x: 1.1 *  M.sw(x) - 4.4 * P(x, Mdual[0]) + 3.3 * P(x, Mdual[1])
adrigid = lambda x: adjoint_outermorphism(rigid, x, alg)
adrigid(rigid(x)), rigid(adrigid(x))

(1.0 𝐞₁ + 2.73 𝐞₂ + -3.77 𝐞₃ + 4.29 𝐞₄, 1.0 𝐞₁ + 2.73 𝐞₂ + -3.77 𝐞₃ + 4.29 𝐞₄)

In [96]:
rigid(x), 3.3 * M**2 * P(x, M.grade(2).normalized()) + 3.3 * P(x, M.grade(2).normalized().dual()) - 4.4 * P(x, Mdual[0]) + 5.5 * P(x, Mdual[1])

(1.0 𝐞₁ + -0.111 𝐞₂ + -2.02 𝐞₃ + 0.637 𝐞₄,
 2.18 𝐞₁ + -1.23 𝐞₂ + -4.16 𝐞₃ + 2.16 𝐞₄ + 1.56e-17 𝐞₁₂₃ + 1.7e-16 𝐞₁₂₄ + -1.8e-16 𝐞₁₃₄ + -1.96e-18 𝐞₂₃₄)

In [97]:
#5.2
def norm(A):
    return np.sqrt(normsq(A))
b1, b2 = norm(F1), norm(F2)
i1, i2 = F1/b1, F2/b2

F, b1*i1 + b2*i2, outermorphism(fm, i1, alg), b1**2*i1

(0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 0.103 𝐞₁₂ + 0.637 𝐞₁₃ + 0.236 𝐞₁₄ + -0.371 𝐞₂₃ + 0.0345 𝐞₂₄ + -0.192 𝐞₃₄,
 0.00611 𝐞₁₂ + -0.00351 𝐞₁₃ + 0.0115 𝐞₁₄ + -0.00566 𝐞₂₃ + 0.0225 𝐞₂₄ + -0.00226 𝐞₃₄,
 0.00611 𝐞₁₂ + -0.00351 𝐞₁₃ + 0.0115 𝐞₁₄ + -0.00566 𝐞₂₃ + 0.0225 𝐞₂₄ + -0.00226 𝐞₃₄)

In [98]:
# 5.3
terms_ratio(outermorphism(fp, i1, alg), i1) # not normal

array([ 3.37213203, -6.12858738, -7.39682306,  3.08525105, -1.16223581,
       43.95772882])

In [99]:
# 5.3
def sym_part(f, alg):
    return lambda x: (f(x) + adjoint_outermorphism(f, x, alg))/2

def skew_part(f, alg):
    return lambda x: (f(x) - adjoint_outermorphism(f, x, alg))/2

R = curl(rigid, x, alg, grade=1)/2
skew_blades = bivector_split(R, alg)
skew_eigs = [norm(b) for b in skew_blades]
skew_blades = [b / e for b,e in zip(skew_blades, skew_eigs) if e > 1e-6]
skew_eigs = [e for e in skew_eigs if e > 1e-6]
rigid_p = sym_part(rigid, alg)
[terms_ratio(outermorphism(rigid_p, i, alg), i) for i in skew_blades]

[array([0.27272809, 0.27272809, 0.27272809, 0.27272809, 0.27272809,
        0.27272809])]

In [100]:
#5.4
rand_vec = create_r_vectors(1, alg)[0]
sym_vecs = [(rand_vec | i).normalized() for i in skew_blades]
other_sym_space = wedge(skew_blades).dual()
sym_eigs = [terms_ratio(rigid_p(a), a)[0] for a in sym_vecs]

sym_eigs, np.square(sym_eigs), [terms_ratio(outermorphism(rigid_p, i, alg), i)[0] for i in skew_blades]


([-0.5222337506945599], array([0.27272809]), [0.27272809041505547])

In [101]:
#5.5a
m = len(skew_blades)
rigid_eigs = [sym_eigs[k] + skew_blades[k] * skew_eigs[k] for k in range(m)]
[rigid(a) for a in sym_vecs], [a*l for a, l in zip(sym_vecs,rigid_eigs)], [l.reverse()*a for a, l in zip(sym_vecs,rigid_eigs)]

([-0.792 𝐞₁ + -0.471 𝐞₂ + -0.536 𝐞₃ + -0.27 𝐞₄],
 [-0.792 𝐞₁ + -0.471 𝐞₂ + -0.536 𝐞₃ + -0.27 𝐞₄ + 5.55e-17 𝐞₁₂₃ + 2.78e-17 𝐞₁₂₄ + 1.39e-17 𝐞₁₃₄ + 1.39e-17 𝐞₂₃₄],
 [-0.792 𝐞₁ + -0.471 𝐞₂ + -0.536 𝐞₃ + -0.27 𝐞₄ + -5.55e-17 𝐞₁₂₃ + -2.78e-17 𝐞₁₂₄ + -1.39e-17 𝐞₁₃₄ + -6.94e-18 𝐞₂₃₄])

In [102]:
#5.5b
[adjoint_outermorphism(rigid, a, alg) for a in sym_vecs], [l*a for a, l in zip(sym_vecs,rigid_eigs)], [a*l.reverse() for a, l in zip(sym_vecs,rigid_eigs)]

([-0.064 𝐞₁ + 1.0 𝐞₂ + 0.446 𝐞₃ + 0.0116 𝐞₄],
 [-0.064 𝐞₁ + 1.0 𝐞₂ + 0.446 𝐞₃ + 0.0116 𝐞₄ + 5.55e-17 𝐞₁₂₃ + 2.78e-17 𝐞₁₂₄ + 1.39e-17 𝐞₁₃₄ + 6.94e-18 𝐞₂₃₄],
 [-0.064 𝐞₁ + 1.0 𝐞₂ + 0.446 𝐞₃ + 0.0116 𝐞₄ + -5.55e-17 𝐞₁₂₃ + -2.78e-17 𝐞₁₂₄ + -1.39e-17 𝐞₁₃₄ + -1.39e-17 𝐞₂₃₄])

In [103]:
[norm(l) for l in rigid_eigs], [np.sqrt(sym_eigs[k] ** 2 + skew_eigs[k] ** 2) for k in range(m)]

([1.0999999999226067], [1.0999999999226069])

In [104]:
#5.6
rho = [norm(l) for l in rigid_eigs] # dilation
theta = [np.arccos(sym_eigs[k] / rho[k]) for k in range(m)] # rotation angle

rigid_eigs, [rho[k] * blade_exp(theta[k] * skew_blades[k]) for k in range(m)] # skew_blades gives rotation plane

([-0.522 + -0.79 𝐞₁₂ + -0.371 𝐞₁₃ + -0.0254 𝐞₁₄ + 0.313 𝐞₂₃ + 0.254 𝐞₂₄ + 0.109 𝐞₃₄],
 [-0.522 + -0.79 𝐞₁₂ + -0.371 𝐞₁₃ + -0.0254 𝐞₁₄ + 0.313 𝐞₂₃ + 0.254 𝐞₂₄ + 0.109 𝐞₃₄])

In [105]:
# spectral decomposition of symmetric part in other_sym_space
other_sym_frame = blade_split(other_sym_space, alg)
other_sym_eigs = np.roots(char_poly_coefs(rigid_p, alg, other_sym_frame, reciprocal(other_sym_frame)))
other_sym_eigs

array([ 3.09783939, -1.99783939])

In [106]:
#5.9 spectral decomposition of a normal transformation
sym_vecs_dual = [a|i for a, i in zip(sym_vecs, skew_blades)]
other_sym_blades = eigenblades_from_values(rigid_p, other_sym_eigs, alg, other_sym_frame)

rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs + sym_vecs_dual + other_sym_blades, rigid_eigs * 2 + list(other_sym_eigs))])

(1.0 𝐞₁ + -0.111 𝐞₂ + -2.02 𝐞₃ + 0.637 𝐞₄,
 1.0 𝐞₁ + -0.111 𝐞₂ + -2.02 𝐞₃ + 0.637 𝐞₄ + 3.47e-18 𝐞₁₂₃ + -1.73e-17 𝐞₁₂₄ + -6.94e-18 𝐞₁₃₄ + 8.67e-18 𝐞₂₃₄)

In [107]:
#5.17
sym_eigs, sym_vecs = np.linalg.eigh(trans2matrix(rigid_p, alg))
sym_vecs = [alg.vector(v) for v in sym_vecs.transpose()]
sym_eigs, sym_vecs

(array([-1.99783939, -0.52223375, -0.52223375,  3.09783939]),
 [0.406 𝐞₁ + 0.196 𝐞₂ + -0.362 𝐞₃ + -0.816 𝐞₄,
  -0.401 𝐞₁ + -0.746 𝐞₂ + -0.51 𝐞₃ + -0.153 𝐞₄,
  0.808 𝐞₁ + -0.532 𝐞₂ + 0.0703 𝐞₃ + 0.243 𝐞₄,
  0.147 𝐞₁ + 0.349 𝐞₂ + -0.777 𝐞₃ + 0.502 𝐞₄])

In [108]:
# In my case, all vectors are both eigenvectors of rigid_p and rigid_n
rigid_n = lambda x: rigid(adjoint_outermorphism(rigid, x, alg))
[terms_ratio(rigid_n(v), v) for v in sym_vecs]

[array([3.99136222, 3.99136222, 3.99136222, 3.99136222]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([1.21, 1.21, 1.21, 1.21]),
 array([9.59660887, 9.59660887, 9.59660887, 9.59660887])]

In [109]:
# the same spectral decomposition of the normal transformation
beta = [v ^ rigid(v) for v in sym_vecs]
rigid_eigs = [a + b for a,b in zip(sym_eigs, beta)]
rigid(x), sum([P(x, a) * l for a, l in zip(sym_vecs, rigid_eigs)])

(1.0 𝐞₁ + -0.111 𝐞₂ + -2.02 𝐞₃ + 0.637 𝐞₄,
 1.0 𝐞₁ + -0.111 𝐞₂ + -2.02 𝐞₃ + 0.637 𝐞₄ + 5.18e-17 𝐞₁₂₃ + -4.02e-18 𝐞₁₂₄ + -2.25e-19 𝐞₁₃₄ + 2.56e-17 𝐞₂₃₄)

In [110]:
#5.19 our function "rigid" is normal but not orthogonal
A, outermorphism(rigid, adjoint_outermorphism(rigid, A, alg, h=1e-2), alg, h=1e-2)

(0.408 + 0.451 𝐞₁ + 0.665 𝐞₂ + 0.0637 𝐞₃ + 0.405 𝐞₄ + 0.143 𝐞₁₂ + 0.631 𝐞₁₃ + 0.737 𝐞₁₄ + 0.503 𝐞₂₃ + 0.683 𝐞₂₄ + 0.599 𝐞₃₄ + 0.225 𝐞₁₂₃ + 0.387 𝐞₁₂₄ + 0.198 𝐞₁₃₄ + 0.233 𝐞₂₃₄ + 0.0946 𝐞₁₂₃₄,
 0.408 + 1.06 𝐞₁ + 2.11 𝐞₂ + -2.83 𝐞₃ + 2.48 𝐞₄ + 0.225 𝐞₁₂ + 4.48 𝐞₁₃ + 1.9 𝐞₁₄ + 2.79 𝐞₂₃ + 1.21 𝐞₂₄ + 6.9 𝐞₃₄ + -1.3 𝐞₁₂₃ + 4.97 𝐞₁₂₄ + 3.44 𝐞₁₃₄ + 12.2 𝐞₂₃₄ + 5.31 𝐞₁₂₃₄)

In [111]:
#5.19 But if we normalize each "eigenvalues" in the spectral decomposition
ortho_eigs = [l.normalized() for l in rigid_eigs]
ortho = lambda x: sum([P(x, a) * l for a, l in zip(sym_vecs, ortho_eigs)])
A, outermorphism(ortho, adjoint_outermorphism(ortho, A, alg, h=1e-2), alg, h=1e-2)

(0.408 + 0.451 𝐞₁ + 0.665 𝐞₂ + 0.0637 𝐞₃ + 0.405 𝐞₄ + 0.143 𝐞₁₂ + 0.631 𝐞₁₃ + 0.737 𝐞₁₄ + 0.503 𝐞₂₃ + 0.683 𝐞₂₄ + 0.599 𝐞₃₄ + 0.225 𝐞₁₂₃ + 0.387 𝐞₁₂₄ + 0.198 𝐞₁₃₄ + 0.233 𝐞₂₃₄ + 0.0946 𝐞₁₂₃₄,
 0.408 + 0.451 𝐞₁ + 0.665 𝐞₂ + 0.0637 𝐞₃ + 0.405 𝐞₄ + 0.143 𝐞₁₂ + 0.631 𝐞₁₃ + 0.737 𝐞₁₄ + 0.503 𝐞₂₃ + 0.683 𝐞₂₄ + 0.599 𝐞₃₄ + 0.225 𝐞₁₂₃ + 0.387 𝐞₁₂₄ + 0.198 𝐞₁₃₄ + 0.233 𝐞₂₃₄ + 0.0946 𝐞₁₂₃₄)

In [112]:
#5.20 left/right product of exponential
ortho(x), sum([l.inv() * P(x, a) for a, l in zip(sym_vecs, ortho_eigs)])

(0.641 𝐞₁ + -0.558 𝐞₂ + -0.836 𝐞₃ + 0.19 𝐞₄ + 1.29e-18 𝐞₁₂₃ + -1.98e-17 𝐞₁₂₄ + 4.73e-19 𝐞₁₃₄ + 1.96e-17 𝐞₂₃₄,
 0.641 𝐞₁ + -0.558 𝐞₂ + -0.836 𝐞₃ + 0.19 𝐞₄ + -2.91e-17 𝐞₁₂₃ + 6.97e-19 𝐞₁₂₄ + 1.12e-17 𝐞₁₃₄ + 2.02e-17 𝐞₂₃₄)

In [113]:
#5.21
(
    [sym_vecs[k-1].normalized() * sym_vecs[k].normalized() for k in range(1, 4)],
    [e.grade(2).normalized() for e in ortho_eigs]
)


([2.22e-16 + -0.224 𝐞₁₂ + -0.352 𝐞₁₃ + -0.389 𝐞₁₄ + -0.37 𝐞₂₃ + -0.639 𝐞₂₄ + -0.361 𝐞₃₄,
  -2.84e-16 + 0.816 𝐞₁₂ + 0.384 𝐞₁₃ + 0.0262 𝐞₁₄ + -0.324 𝐞₂₃ + -0.262 𝐞₂₄ + -0.113 𝐞₃₄,
  -4.16e-17 + 0.361 𝐞₁₂ + -0.639 𝐞₁₃ + 0.37 𝐞₁₄ + 0.389 𝐞₂₃ + -0.352 𝐞₂₄ + 0.224 𝐞₃₄],
 [-0.0555 𝐞₁₂ + -0.45 𝐞₁₃ + 0.0423 𝐞₁₄ + -0.225 𝐞₂₃ + -0.0847 𝐞₂₄ + -0.857 𝐞₃₄,
  -0.816 𝐞₁₂ + -0.384 𝐞₁₃ + -0.0262 𝐞₁₄ + 0.324 𝐞₂₃ + 0.262 𝐞₂₄ + 0.113 𝐞₃₄,
  -0.816 𝐞₁₂ + -0.384 𝐞₁₃ + -0.0262 𝐞₁₄ + 0.324 𝐞₂₃ + 0.262 𝐞₂₄ + 0.113 𝐞₃₄,
  0.0207 𝐞₁₂ + 0.196 𝐞₁₃ + 0.0273 𝐞₁₄ + 0.588 𝐞₂₃ + -0.000941 𝐞₂₄ + -0.784 𝐞₃₄])

In [114]:
#5.23
def rotor_sqrt(R):
    R_norm = norm(R+1)
    assert R_norm >= 1e-4, "no explicit square root for -1"
    return (R + 1)/R_norm
    
neg_eig_pairs = []
eig_rotors = []
for i, e in enumerate(ortho_eigs):
    if normsq(e + 1) <= 1e-4:
        neg_eig_pairs.append((-1, sym_vecs[i]))
    else:
        eig_rotors.append(rotor_sqrt(e))

sym_vecs, [[e.reverse().sw(a) for e in eig_rotors] for a in sym_vecs]

([0.406 𝐞₁ + 0.196 𝐞₂ + -0.362 𝐞₃ + -0.816 𝐞₄,
  -0.401 𝐞₁ + -0.746 𝐞₂ + -0.51 𝐞₃ + -0.153 𝐞₄,
  0.808 𝐞₁ + -0.532 𝐞₂ + 0.0703 𝐞₃ + 0.243 𝐞₄,
  0.147 𝐞₁ + 0.349 𝐞₂ + -0.777 𝐞₃ + 0.502 𝐞₄],
 [[0.406 𝐞₁ + 0.196 𝐞₂ + -0.362 𝐞₃ + -0.816 𝐞₄,
   0.406 𝐞₁ + 0.196 𝐞₂ + -0.362 𝐞₃ + -0.816 𝐞₄,
   0.406 𝐞₁ + 0.196 𝐞₂ + -0.362 𝐞₃ + -0.816 𝐞₄],
  [-0.521 𝐞₁ + 0.822 𝐞₂ + 0.18 𝐞₃ + -0.141 𝐞₄,
   -0.521 𝐞₁ + 0.822 𝐞₂ + 0.18 𝐞₃ + -0.141 𝐞₄,
   -0.401 𝐞₁ + -0.746 𝐞₂ + -0.51 𝐞₃ + -0.153 𝐞₄],
  [-0.736 𝐞₁ + -0.404 𝐞₂ + -0.482 𝐞₃ + -0.25 𝐞₄,
   -0.736 𝐞₁ + -0.404 𝐞₂ + -0.482 𝐞₃ + -0.25 𝐞₄,
   0.808 𝐞₁ + -0.532 𝐞₂ + 0.0703 𝐞₃ + 0.243 𝐞₄],
  [0.147 𝐞₁ + 0.349 𝐞₂ + -0.777 𝐞₃ + 0.502 𝐞₄,
   0.147 𝐞₁ + 0.349 𝐞₂ + -0.777 𝐞₃ + 0.502 𝐞₄,
   0.147 𝐞₁ + 0.349 𝐞₂ + -0.777 𝐞₃ + 0.502 𝐞₄]])

In [115]:
#5.25 I selected an example with -1 as an eigenvalue, 
# so we need an extra fix for the canonical form.
theta = reduce(alg.gp, eig_rotors[::2])

ortho(x), theta.reverse().sw(x) - (2*sum(P(x, neg_eig_pairs[i][1]) for i in range(len(neg_eig_pairs))))

(0.641 𝐞₁ + -0.558 𝐞₂ + -0.836 𝐞₃ + 0.19 𝐞₄ + 1.29e-18 𝐞₁₂₃ + -1.98e-17 𝐞₁₂₄ + 4.73e-19 𝐞₁₃₄ + 1.96e-17 𝐞₂₃₄,
 0.641 𝐞₁ + -0.558 𝐞₂ + -0.836 𝐞₃ + 0.19 𝐞₄)